In [1]:
# RUN ONCE AND ONCE ONLY PER SESSION
import dask
from dask.distributed import Client, LocalCluster

try:
    client = Client('127.0.0.1:8786')
except:
    cluster = LocalCluster(scheduler_port=8786)
    client = Client(cluster)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:8786,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 31.73 GiB
Comm: tcp://127.0.0.1:57652,Total threads: 4
Dashboard: http://127.0.0.1:57655/status,Memory: 7.93 GiB
Nanny: tcp://127.0.0.1:57625,


In [3]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: prophet in c:\users\trainor-gilhamt\anaconda3\lib\site-packages (1.1)



In [4]:
!pip install --upgrade plotly

In [5]:
# Run as is
import pandas as pd
import numpy as np
import datetime
import glob, os
from collections import defaultdict
from multiprocessing import Pool, cpu_count
from functools import partial

from prophet import Prophet
from prophet.plot import plot_plotly, plot_cross_validation_metric
from prophet.make_holidays import make_holidays_df
from prophet.diagnostics import cross_validation, performance_metrics

from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, LabelEncoder

import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
# Change the value in this bit of code "country_name = 'ZA'"
# This should reflect the country you are running the curve for
# Use the Alpha 3 code from this list: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes

class ProphetPredictions:
    def __init__(self, model, figforecast, figcomponents, dfcrossvalidation, dfperformancemetrics, figcrossvalidation):
        self.model = model
        self.figforecast = figforecast
        self.figcomponents = figcomponents
        self.dfcrossvalidation = dfcrossvalidation
        self.dfperformancemetrics = dfperformancemetrics
        self.figcrossvalidation = figcrossvalidation

    def getmodel(self):
        return self.model


def mapefn(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def generate_prediction(myhols, dfpredict):
    #mdl = Prophet(seasonality_mode='multiplicative', seasonality_prior_scale=0.5)    
    ## add cap & floor for df & future
    mydf = dfpredict.copy()
    # Box-Cox transformation of Order 0
    mydf['y'] = np.log(1+mydf['y']) #log transform
    mydf['cap'] = mydf['y'].max()
    mydf['floor'] = 1
    
    mdl = Prophet(growth='logistic', changepoint_prior_scale=0.001, holidays=myhols, yearly_seasonality=True).fit(mydf, iter=100)
    
    # Create future dataset & forecast against
    future = mdl.make_future_dataframe(periods=52, freq='W')
    future['cap'] = mydf['y'].max()
    future['floor'] = 1
    #Maybe update hols here?

    # Predict
    forecast = mdl.predict(future)
    
    # Inverse Box-Cox
    forecast['yhat'] = np.exp(forecast['yhat']) - 1
    forecast['trend'] = np.exp(forecast['trend']) - 1

    # print("Calculating MAPE . . .")
    # df_mape = mapefn(dfpredict['y'], forecast['yhat'])

    return forecast  # ,df_mape,



# Use the Alpha 3 code from this list: https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes
def makeprophethols(timesdf, future_periods, date_freq):
    # Make Holidays with bounds
    year_list = [2020, 2021, 2022]
    country_name = 'NLD'
    myhols = make_holidays_df(year_list, country_name)

    # Create all dates
    myidx = timesdf.sort_values()
    myidx = myidx.drop_duplicates()
    firstdate = myidx[0]
    lastdate = myidx[-1]
    future_dates = pd.date_range(start=lastdate, periods=future_periods+1, freq=date_freq)
    future_dates = future_dates[future_dates > lastdate] # Drop start date if equal to lastdate
    myidx = myidx.append(future_dates[:future_periods]) # Return correct number of periods
    lastdate = myidx[-1] # Update last date
    
    # Select only holidays beginning from 7 days prior to start of dataset
    myhols = myhols.loc[myhols['ds'] > (firstdate - pd.offsets.Day(7))]

    # Select only holidays beginning from 7 days prior to start of dataset to ending last date of dataset
    myhols = myhols.loc[(myhols['ds'] > (firstdate - pd.offsets.Day(7))) & (myhols['ds'] <= lastdate)]
    myhols.reset_index(inplace=True, drop=True)

    # Finds the closest date for each holiday that can be found in your current dataset
    for i in range(len(myhols)):
        myhols.loc[i, 'refdate'] = myidx[myidx.get_loc(pd.to_datetime(myhols.loc[i, 'ds']), method='nearest')]


    myhols['datediff'] = myhols['refdate'] - myhols['ds']

    # Additional time delta ensures it runs up to end of month which is needed when forecasting future monthly data
    for i in range(len(myhols)):
        if myhols.loc[i, 'datediff'] >= datetime.timedelta(minutes=0):
            myhols.loc[i, 'upper_window'] = pd.to_timedelta(myhols.loc[i, 'refdate'] - myhols.loc[i, 'ds'])
        else:
            myhols.loc[i, 'upper_window'] = 0

    for i in range(len(myhols)):
        if myhols.loc[i, 'datediff'] < datetime.timedelta(minutes=0):
            myhols.loc[i, 'lower_window'] = pd.to_timedelta(myhols.loc[i, 'refdate'] - myhols.loc[i, 'ds'])
        else:
            myhols.loc[i, 'lower_window'] = 0

    myhols['upper_window'] = pd.to_timedelta(myhols['upper_window']).dt.days
    myhols['lower_window'] = pd.to_timedelta(myhols['lower_window']).dt.days
    myhols.drop(columns=['refdate', 'datediff'], inplace=True)
    # Set window for the future dataframe
    #To be fixed/created. Ignore future timeperiods for now
    return myhols

def generateweeklyaggs(df, valcol, distcol):
    df['val_pp1'] = df[valcol].shift(periods=1)
    df['dist_pp1'] = df[distcol].shift(periods=1)

    df['val_pp2'] = df[valcol].shift(periods=2)
    df['dist_pp2'] = df[distcol].shift(periods=2)

    df['val_pp3'] = df[valcol].shift(periods=3)
    df['dist_pp3'] = df[distcol].shift(periods=3)

    df['val_chg1'] = df[valcol].diff(periods=1)
    df['dist_chg1'] = df[distcol].diff(periods=1)

    df['val_chg2'] = df[valcol].diff(periods=2)
    df['dist_chg2'] = df[distcol].diff(periods=2)

    df['val_chg3'] = df[valcol].diff(periods=3)
    df['dist_chg3'] = df[distcol].diff(periods=3)

    df['val4wk'] = df[valcol].rolling(4).sum()
    df['dist4wk'] = df[distcol].rolling(4).sum()

    df['val_4wkchg1'] = df['val4wk'].diff(periods=1)
    df['dist_4wkchg1'] = df['dist4wk'].diff(periods=1)

    df['val_4wkchg2'] = df['val4wk'].diff(periods=2)
    df['dist_4wkchg2'] = df['dist4wk'].diff(periods=2)

    return df


def outlier_week_check(row, valchg, val_pp):
    ##Returns true if the % change vs. prev week is greater than 100%
    return (abs(row[valchg] / row[val_pp]) // 1) >= 1


def calculatesupplementarymetrics(df, valcol, distcol):
    df['val_chg1_perc'] = df['val_chg1'] / df['val_pp1']
    df['dist_chg1_perc'] = df['dist_chg1'] / df['dist_pp1']

    df['val_chg2_perc'] = df['val_chg2'] / df['val_pp3']
    df['dist_chg2_perc'] = df['dist_chg2'] / df['dist_pp3']

    df['val_4wkchg1_perc'] = df['val_4wkchg1'] / (df['val4wk'] + df['val_4wkchg1'])
    df['dist_4wkchg1_perc'] = df['dist_4wkchg1'] / (df['dist4wk'] + df['dist_4wkchg1'])

    df['val_4wkchg2_perc'] = df['val_4wkchg2'] / (
            df['val4wk'] + df['val_4wkchg1'] + df['val_4wkchg2'])
    df['dist_4wkchg2_perc'] = df['dist_4wkchg2'] / (
            df['dist4wk'] + df['dist_4wkchg1'] + df['dist_4wkchg2'])

    df.replace((np.inf, -np.inf), (0, 0), inplace=True)

    return df


def createfile(df_input, valcol, tdpcol):
    # Drop zero rows to avoid divide by zero errors
    df = df_input.copy()
    df.drop(df[df[valcol] == 0].index, inplace=True)
    df.drop(df[df[tdpcol] <= 0].index, inplace=True)

    df = generateweeklyaggs(df, valcol, tdpcol)
    df = calculatesupplementarymetrics(df, valcol, tdpcol)

    # Remove rows with less than 4weeks prior history
    df.drop(df[df['dist_chg1'].isna()].index, inplace=True)

    df['is_outlier(100%chg)'] = df.apply(outlier_week_check, axis=1, args=('val_chg1', 'val_pp1')).astype('int')
    return df

Change the name of the two csv files you will load in. First, the sales file/raw data and secondly, the segmentation file saved as a csv

In [7]:
my_path = r"C:\Users\trainor-gilhamt\Documents\Unilever NL\SoftDrinks\ISD\\"
files_list = glob.glob(my_path+"*.csv")
tt_raw_dict = {}
for f in files_list:
    temp_df = pd.DataFrame()
    temp_df = pd.read_csv(f, dtype={"Barcode":str})
    key_value = f.replace(my_path,"").replace(".csv","")
    temp_df['Region'] = key_value
    tt_raw_dict[key_value] = temp_df
    
#tt_raw_list = pd.concat([pd.read_csv(f, dtype={"Barcode":str}) for f in files_list])
tt_raw_dict.keys()

dict_keys(['C:\\Users\\trainor-gilhamt\\Documents\\Unilever NL\\SoftDrinks\\ISD\\UL_NL_SoftDrinks_Curve_Model_Data_Transformed'])

In [8]:
tt_raw_orig = pd.concat([tt_raw_dict[f] for f in tt_raw_dict.keys()])
tt_raw_orig.describe()

,UPC_DESC,UNITS,SALES (EUR),ACV
count,"594,392.00","594,392.00","594,392.00","594,392.00"
mean,"46,782,916.95","2,918.96","4,030.34",52.26
std,"25,107,302.03","10,555.18","14,778.94",41.93
min,"6,797.00",0.04,0.01,0.00
25%,"23,991,688.00",14.00,21.27,3.16
50%,"53,703,360.00",394.00,509.03,59.68
75%,"70,136,330.00","2,047.00","2,762.89",96.19
max,"85,360,903.00","609,459.00","740,872.50",100.00


In [9]:
tt_raw = tt_raw_orig.loc[(tt_raw_orig['ACV'] <= 100) & (tt_raw_orig['ACV'] > 0) & (tt_raw_orig['UNITS'] > 0) & (tt_raw_orig['SALES (EUR)'] > 0)].reset_index(drop=True)
tt_raw.describe()

,UPC_DESC,UNITS,SALES (EUR),ACV
count,"594,392.00","594,392.00","594,392.00","594,392.00"
mean,"46,782,916.95","2,918.96","4,030.34",52.26
std,"25,107,302.03","10,555.18","14,778.94",41.93
min,"6,797.00",0.04,0.01,0.00
25%,"23,991,688.00",14.00,21.27,3.16
50%,"53,703,360.00",394.00,509.03,59.68
75%,"70,136,330.00","2,047.00","2,762.89",96.19
max,"85,360,903.00","609,459.00","740,872.50",100.00


In [10]:
tt_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594392 entries, 0 to 594391
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   RETAILER     594392 non-null  object 
 1   UPC_DESC     594392 non-null  int64  
 2   WEEK         594392 non-null  object 
 3   UNITS        594392 non-null  float64
 4   SALES (EUR)  594392 non-null  float64
 5   ACV          594392 non-null  float64
 6   Region       594392 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 31.7+ MB


In [11]:
tt_raw

,RETAILER,UPC_DESC,WEEK,UNITS,SALES (EUR),ACV,Region
0,Albert Heijn,7087,2020-11-29,1.00,3.39,0.05,C:\Users\trainor-gilhamt\Documents\Unilever NL...
1,Albert Heijn,7087,2020-12-13,1.00,3.39,0.06,C:\Users\trainor-gilhamt\Documents\Unilever NL...
2,Albert Heijn,7087,2021-10-24,1.00,5.25,0.10,C:\Users\trainor-gilhamt\Documents\Unilever NL...
3,Albert Heijn,7202,2021-08-01,1.00,5.39,0.07,C:\Users\trainor-gilhamt\Documents\Unilever NL...
4,Albert Heijn,7237,2020-11-22,1.00,0.85,0.04,C:\Users\trainor-gilhamt\Documents\Unilever NL...
...,...,...,...,...,...,...,...
594387,Vomar,84759825,2022-08-21,4.00,8.07,2.93,C:\Users\trainor-gilhamt\Documents\Unilever NL...
594388,Vomar,84759825,2022-09-04,1.00,1.34,2.56,C:\Users\trainor-gilhamt\Documents\Unilever NL...
594389,Vomar,84759825,2022-09-11,3.00,4.04,2.58,C:\Users\trainor-gilhamt\Documents\Unilever NL...
594390,Vomar,85359716,2022-10-02,39.00,89.54,20.50,C:\Users\trainor-gilhamt\Documents\Unilever NL...


In [12]:
tt_raw['RETAILER'].unique()

array(['Albert Heijn', 'Hoogvliet', 'Jumbo', 'Plus', 'Vomar'],
      dtype=object)

In [13]:
tt_raw['WEEK'].unique()

array(['2020-11-29', '2020-12-13', '2021-10-24', '2021-08-01',
       '2020-11-22', '2020-12-20', '2021-01-03', '2021-01-24',
       '2021-01-31', '2021-03-07', '2021-03-14', '2021-03-21',
       '2021-03-28', '2021-04-25', '2021-05-16', '2021-06-06',
       '2021-06-13', '2021-06-20', '2021-06-27', '2021-07-04',
       '2021-07-11', '2021-07-18', '2021-07-25', '2021-08-08',
       '2021-08-15', '2021-08-22', '2021-08-29', '2021-09-05',
       '2021-09-12', '2021-09-26', '2021-10-03', '2021-11-14',
       '2021-12-12', '2022-01-09', '2022-03-06', '2022-03-27',
       '2022-05-01', '2022-05-08', '2022-05-15', '2022-05-22',
       '2022-08-21', '2022-09-18', '2022-08-28', '2020-11-01',
       '2021-05-09', '2021-10-10', '2021-10-17', '2020-10-11',
       '2021-01-10', '2021-01-17', '2021-09-19', '2022-04-03',
       '2020-12-06', '2022-09-25', '2022-10-02', '2022-10-09',
       '2020-10-18', '2020-10-25', '2020-11-15', '2020-12-27',
       '2021-02-07', '2021-02-14', '2021-02-21', '2021-

In [14]:
tt_raw['WEEK'] = pd.to_datetime(tt_raw['WEEK'])
tt_raw['WEEK'].unique()

array(['2020-11-29T00:00:00.000000000', '2020-12-13T00:00:00.000000000',
       '2021-10-24T00:00:00.000000000', '2021-08-01T00:00:00.000000000',
       '2020-11-22T00:00:00.000000000', '2020-12-20T00:00:00.000000000',
       '2021-01-03T00:00:00.000000000', '2021-01-24T00:00:00.000000000',
       '2021-01-31T00:00:00.000000000', '2021-03-07T00:00:00.000000000',
       '2021-03-14T00:00:00.000000000', '2021-03-21T00:00:00.000000000',
       '2021-03-28T00:00:00.000000000', '2021-04-25T00:00:00.000000000',
       '2021-05-16T00:00:00.000000000', '2021-06-06T00:00:00.000000000',
       '2021-06-13T00:00:00.000000000', '2021-06-20T00:00:00.000000000',
       '2021-06-27T00:00:00.000000000', '2021-07-04T00:00:00.000000000',
       '2021-07-11T00:00:00.000000000', '2021-07-18T00:00:00.000000000',
       '2021-07-25T00:00:00.000000000', '2021-08-08T00:00:00.000000000',
       '2021-08-15T00:00:00.000000000', '2021-08-22T00:00:00.000000000',
       '2021-08-29T00:00:00.000000000', '2021-09-05

In [15]:
salesdf_raw = tt_raw[['RETAILER','UPC_DESC','WEEK','UNITS','SALES (EUR)','ACV']].copy()
#salesdf_raw = tt.copy()
salesdf_raw

,RETAILER,UPC_DESC,WEEK,UNITS,SALES (EUR),ACV
0,Albert Heijn,7087,2020-11-29,1.00,3.39,0.05
1,Albert Heijn,7087,2020-12-13,1.00,3.39,0.06
2,Albert Heijn,7087,2021-10-24,1.00,5.25,0.10
3,Albert Heijn,7202,2021-08-01,1.00,5.39,0.07
4,Albert Heijn,7237,2020-11-22,1.00,0.85,0.04
...,...,...,...,...,...,...
594387,Vomar,84759825,2022-08-21,4.00,8.07,2.93
594388,Vomar,84759825,2022-09-04,1.00,1.34,2.56
594389,Vomar,84759825,2022-09-11,3.00,4.04,2.58
594390,Vomar,85359716,2022-10-02,39.00,89.54,20.50


In [16]:
# Load in your data file
#salesdf_raw = pd.read_csv('C:\\Users\\kuria\\Downloads\\Data Transform\\Asahi UK\\HANDS RAWDATA JAN 20.csv', header = 0, thousands = ',', error_bad_lines = True, warn_bad_lines = True)
segdf = pd.read_excel(my_path+'SEG_PL_Template_UL_NL_Soft Drinks_Nov 22.xlsx',  sheet_name = 'seg')
segdf

,UPC_DESC,CATEGORY,SEG_DESC,SUB_SEG_DESC,BASE_DESC,BASE_SIZE,MANUFACTURER,BRAND,US_UNITS,US_ACV,US_DOLLARS,ProductDescription,FRISDRANKEN_NAAR_SORT_VARI_LEVEL_0,SUBSOORT,VERPAKKINGSVORM,BASISINHOUD,ProductAttribute5,ProductAttribute6,ProductAttribute7,ProductAttribute8,ProductAttribute9,ProductAttribute10,ProductAttribute11,ProductAttribute12,ProductAttribute13,ProductAttribute14,ProductAttribute15,ProductAttribute16,ProductAttribute17,ProductAttribute18,ProductAttribute19,ProductAttribute20
0,63898203,Soft Drinks 22,AO SEGMENTS,BITTER LEMON - REGULAR - EIGEN MERK,63898203,X1,EIGEN,E.M.,"96,617.00",0.11,"39,588.81",E.M. BITTER_LE KZH REG. 250_ML,TOT. FRISDRANKEN,BITTER_LEM,BLIK,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63898232,Soft Drinks 22,AO SEGMENTS,BITTER LEMON - REGULAR - EIGEN MERK,63898232,X1,EIGEN,E.M.,"103,141.00",0.13,"42,273.47",E.M. GINGER KZH REG. 250_ML,TOT. FRISDRANKEN,BITTER_LEM,BLIK,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42180929,Soft Drinks 22,AO SEGMENTS,BITTER LEMON - REGULAR - ROYAL CLUB & AO BRANDS,42180929,X1,RED_BULL,ORGNCS_BY_RD_BLL,NaN,0.00,NaN,ORGNCS_BY BITTER_LE KZH REG. 250_ML,TOT. FRISDRANKEN,BITTER_LEM,BLIK,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62779370,Soft Drinks 22,AO SEGMENTS,BITTER LEMON - REGULAR - ROYAL CLUB & AO BRANDS,62779370,X1,USD,LONDON,"40,336.00",0.03,"16,372.08",LONDON BITTER_LE KZH REG. 250_ML,TOT. FRISDRANKEN,BITTER_LEM,BLIK,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71323594,Soft Drinks 22,AO SEGMENTS,BITTER LEMON - LIGHT,71323594,X1,VRUMONA,ROYAL_CLUB,"747,908.84",0.51,"450,486.19",ROYAL_CLU BITTER_LE KZH LIGH 250_ML,TOT. FRISDRANKEN,BITTER_LEM,BLIK,250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8499,42589161,Soft Drinks 22,ENERGY - AO BRANDS,NOCCO,42589161,X1,NOCCO,NOCCO,NaN,0.00,NaN,NOCCO PEER KZH LIGH 330_ML,TOT. FRISDRANKEN,ENERGY,BLIK,330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8500,42588914,Soft Drinks 22,ENERGY - AO BRANDS,NOCCO,42588914,X1,NOCCO,NOCCO,"30,604.00",0.07,"55,567.82",NOCCO PERZIK KZH LIGH 330_ML,TOT. FRISDRANKEN,ENERGY,BLIK,330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8501,42589278,Soft Drinks 22,ENERGY - AO BRANDS,NOCCO,42589278,X1,NOCCO,NOCCO,27.00,0.00,61.76,NOCCO TROPICAL KZH LIGH 330_ML,TOT. FRISDRANKEN,ENERGY,BLIK,330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8502,79548130,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - SPA,79548130,X6,SPADEL,SPA,123.00,0.00,"1,069.02",SPA CITR KZH REG. 7500_ML_6_,TOT. FRISDRANKEN,MIN.W.CITR.,FLES_GROOT,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Run as is
segdf.columns

Index(['UPC_DESC', 'CATEGORY', 'SEG_DESC', 'SUB_SEG_DESC', 'BASE_DESC',
       'BASE_SIZE', 'MANUFACTURER', 'BRAND', 'US_UNITS', 'US_ACV',
       'US_DOLLARS', 'ProductDescription',
       'FRISDRANKEN_NAAR_SORT_VARI_LEVEL_0', 'SUBSOORT', 'VERPAKKINGSVORM',
       'BASISINHOUD', 'ProductAttribute5', 'ProductAttribute6',
       'ProductAttribute7', 'ProductAttribute8', 'ProductAttribute9',
       'ProductAttribute10', 'ProductAttribute11', 'ProductAttribute12',
       'ProductAttribute13', 'ProductAttribute14', 'ProductAttribute15',
       'ProductAttribute16', 'ProductAttribute17', 'ProductAttribute18',
       'ProductAttribute19', 'ProductAttribute20'],
      dtype='object')

Edit the left hand side of the colon which should match the list of columns above

Ensure you replace "Barcode" below with the name of your UPC column as per above and so on

In [18]:
# optional
segdf.rename(columns={'UPC_DESC': 'UPC',
                     }, inplace=True)

Ensure the output of the table below:

1) Has data that is in line with your expectations e.g. Mean & Standard Deviation

2) Has combined both raw sales data & segmentation data files

In [19]:
salesdf_raw.describe()

,UPC_DESC,UNITS,SALES (EUR),ACV
count,"594,392.00","594,392.00","594,392.00","594,392.00"
mean,"46,782,916.95","2,918.96","4,030.34",52.26
std,"25,107,302.03","10,555.18","14,778.94",41.93
min,"6,797.00",0.04,0.01,0.00
25%,"23,991,688.00",14.00,21.27,3.16
50%,"53,703,360.00",394.00,509.03,59.68
75%,"70,136,330.00","2,047.00","2,762.89",96.19
max,"85,360,903.00","609,459.00","740,872.50",100.00


If your file has multiple values OR Units columns e.g. Base & Total, drop the type you don't need

The code below drops the Total type as the base column has sufficient rows & correct info to use instead of the totals

Replace UNITS, DOLLARS with the name of the columns you will drop

In [20]:
# Run as is
salesdf_raw.columns

Index(['RETAILER', 'UPC_DESC', 'WEEK', 'UNITS', 'SALES (EUR)', 'ACV'], dtype='object')

In [21]:
# If you do have extra columns to drop, uncomment the line below and run
#salesdf_raw.drop(columns=['Product_Description','MARKET'], inplace=True)
#salesdf_raw.drop(columns=['WEEK'], inplace=True)

Ensure you replace "BASE_UNITS" with the name of your Units column as per above and so on where needed
Only edit the left hand side of the colon which should match the list of columns above

In [22]:
salesdf = salesdf_raw.rename(columns={'WEEK':'ds', 
                                      'UPC_DESC': 'UPC',
                                      'RETAILER':'Retailer',
                                      'UNITS':'UNITS',
                                      'SALES (EUR)':'VALUE',
                                      'ACV':'ACV'
                                     }
                            )

# Ensures matches will work as expected via string matching
salesdf['UPC'] = salesdf['UPC'].astype(str)
segdf['UPC'] = segdf['UPC'].astype(str)

salesdf = salesdf.merge(segdf, on='UPC')
# Find all non-UPC seg columns and rename them
for col in salesdf.columns[salesdf.columns.isin(segdf.columns[~segdf.columns.str.contains('UPC')])]:
    salesdf.rename(columns = {col: col+'_ATTRIBUTE'}, inplace=True)
    
salesdf

,Retailer,UPC,ds,UNITS,VALUE,ACV,CATEGORY_ATTRIBUTE,SEG_DESC_ATTRIBUTE,SUB_SEG_DESC_ATTRIBUTE,BASE_DESC_ATTRIBUTE,BASE_SIZE_ATTRIBUTE,MANUFACTURER_ATTRIBUTE,BRAND_ATTRIBUTE,US_UNITS_ATTRIBUTE,US_ACV_ATTRIBUTE,US_DOLLARS_ATTRIBUTE,ProductDescription_ATTRIBUTE,FRISDRANKEN_NAAR_SORT_VARI_LEVEL_0_ATTRIBUTE,SUBSOORT_ATTRIBUTE,VERPAKKINGSVORM_ATTRIBUTE,BASISINHOUD_ATTRIBUTE,ProductAttribute5_ATTRIBUTE,ProductAttribute6_ATTRIBUTE,ProductAttribute7_ATTRIBUTE,ProductAttribute8_ATTRIBUTE,ProductAttribute9_ATTRIBUTE,ProductAttribute10_ATTRIBUTE,ProductAttribute11_ATTRIBUTE,ProductAttribute12_ATTRIBUTE,ProductAttribute13_ATTRIBUTE,ProductAttribute14_ATTRIBUTE,ProductAttribute15_ATTRIBUTE,ProductAttribute16_ATTRIBUTE,ProductAttribute17_ATTRIBUTE,ProductAttribute18_ATTRIBUTE,ProductAttribute19_ATTRIBUTE,ProductAttribute20_ATTRIBUTE
0,Albert Heijn,7087,2020-11-29,1.00,3.39,0.05,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - AO BRANDS,7087,X6,SPADEL,BRU,1.00,0.00,5.25,BRU N.SMAAK KZH MIN. 7500_ML_6_,TOT. FRISDRANKEN,MINERAALWATER,FLES_GROOT,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albert Heijn,7087,2020-12-13,1.00,3.39,0.06,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - AO BRANDS,7087,X6,SPADEL,BRU,1.00,0.00,5.25,BRU N.SMAAK KZH MIN. 7500_ML_6_,TOT. FRISDRANKEN,MINERAALWATER,FLES_GROOT,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albert Heijn,7087,2021-10-24,1.00,5.25,0.10,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - AO BRANDS,7087,X6,SPADEL,BRU,1.00,0.00,5.25,BRU N.SMAAK KZH MIN. 7500_ML_6_,TOT. FRISDRANKEN,MINERAALWATER,FLES_GROOT,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albert Heijn,7202,2021-08-01,1.00,5.39,0.07,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - SPA,7202,X6,SPADEL,SPA,NaN,0.00,NaN,SPA N.SMAAK KZH MIN. 9000_ML_6_,TOT. FRISDRANKEN,MINERAALWATER,FLES_GROOT,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albert Heijn,7237,2020-11-22,1.00,0.85,0.04,Soft Drinks 22,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,PET MULTISERVE - KZH - SPA,7237,X1,SPADEL,SPA,"5,250.00",0.01,"5,544.43",SPA N.SMAAK KZH MIN. 1500_ML,TOT. FRISDRANKEN,MINERAALWATER,FLES_GROOT,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593089,Vomar,83212213,2022-09-11,982.00,971.98,98.78,Soft Drinks 22,VRUCHTENDRANK - EIGEN MERK,PET MULTISERVE,83212213,X1,EIGEN,E.M.,"21,672.44",0.02,"24,040.16",E.M. APP_CTR_G N.DR 1000_ML,TOT. FRISDRANKEN,VRUCHTENDRANK,FLES_GROOT,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593090,Vomar,83212213,2022-09-18,973.00,963.17,97.49,Soft Drinks 22,VRUCHTENDRANK - EIGEN MERK,PET MULTISERVE,83212213,X1,EIGEN,E.M.,"21,672.44",0.02,"24,040.16",E.M. APP_CTR_G N.DR 1000_ML,TOT. FRISDRANKEN,VRUCHTENDRANK,FLES_GROOT,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593091,Vomar,83212213,2022-09-25,"1,016.00","1,005.54",97.35,Soft Drinks 22,VRUCHTENDRANK - EIGEN MERK,PET MULTISERVE,83212213,X1,EIGEN,E.M.,"21,672.44",0.02,"24,040.16",E.M. APP_CTR_G N.DR 1000_ML,TOT. FRISDRANKEN,VRUCHTENDRANK,FLES_GROOT,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
593092,Vomar,83212213,2022-10-02,"1,076.00","1,065.04",98.57,Soft Drinks 22,VRUCHTENDRANK - EIGEN MERK,PET MULTISERVE,83212213,X1,EIGEN,E.M.,"21,672.44",0.02,"24,040.16",E.M. APP_CTR_G N.DR 1000_ML,TOT. FRISDRANKEN,VRUCHTENDRANK,FLES_GROOT,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fill in the two variables below (ANALYSIS_CATEGORY & FOCUS_ATTRIBUTE_COLUMN) with the appropriate value

In [23]:
salesdf.columns

Index(['Retailer', 'UPC', 'ds', 'UNITS', 'VALUE', 'ACV', 'CATEGORY_ATTRIBUTE',
       'SEG_DESC_ATTRIBUTE', 'SUB_SEG_DESC_ATTRIBUTE', 'BASE_DESC_ATTRIBUTE',
       'BASE_SIZE_ATTRIBUTE', 'MANUFACTURER_ATTRIBUTE', 'BRAND_ATTRIBUTE',
       'US_UNITS_ATTRIBUTE', 'US_ACV_ATTRIBUTE', 'US_DOLLARS_ATTRIBUTE',
       'ProductDescription_ATTRIBUTE',
       'FRISDRANKEN_NAAR_SORT_VARI_LEVEL_0_ATTRIBUTE', 'SUBSOORT_ATTRIBUTE',
       'VERPAKKINGSVORM_ATTRIBUTE', 'BASISINHOUD_ATTRIBUTE',
       'ProductAttribute5_ATTRIBUTE', 'ProductAttribute6_ATTRIBUTE',
       'ProductAttribute7_ATTRIBUTE', 'ProductAttribute8_ATTRIBUTE',
       'ProductAttribute9_ATTRIBUTE', 'ProductAttribute10_ATTRIBUTE',
       'ProductAttribute11_ATTRIBUTE', 'ProductAttribute12_ATTRIBUTE',
       'ProductAttribute13_ATTRIBUTE', 'ProductAttribute14_ATTRIBUTE',
       'ProductAttribute15_ATTRIBUTE', 'ProductAttribute16_ATTRIBUTE',
       'ProductAttribute17_ATTRIBUTE', 'ProductAttribute18_ATTRIBUTE',
       'ProductAttribute19

In [24]:
# This is the name of your category & name-prefix of the output files
ANALYSIS_CATEGORY = 'UL_NL_SD_ISD'
# This is the attribute you are creating an attribute specific curve for
# If you're running ISD, it is usually SEG_DESC_ATTRIBUTE
FOCUS_ATTRIBUTE_COLUMN = 'SEG_DESC_ATTRIBUTE'

print(f"There are {salesdf[FOCUS_ATTRIBUTE_COLUMN].nunique()} unique values in {FOCUS_ATTRIBUTE_COLUMN}\n The relative shares are as below:")
salesdf[[FOCUS_ATTRIBUTE_COLUMN, 'VALUE']].groupby(FOCUS_ATTRIBUTE_COLUMN).sum()/salesdf['VALUE'].sum()

There are 43 unique values in SEG_DESC_ATTRIBUTE
 The relative shares are as below:


,VALUE
SEG_DESC_ATTRIBUTE,
AO SEGMENTS,0.01
COLA - LIGHT - COCA COLA & AO BRANDS,0.02
COLA - LIGHT - EIGEN MERK,0.02
COLA - LIGHT - PEPSI,0.02
COLA - REGULAR - COCA COLA,0.12
COLA - REGULAR - EIGEN MERK,0.02
COLA - REGULAR - PEPSI & AO BRANDS,0.01
COLA - ZERO - COCA COLA,0.09
ENERGY - AO BRANDS,0.06


In [25]:
# Run as is
# This creates the working dataframe & deletes rows with NA values
forecastdf = salesdf[[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds','UNITS','VALUE','ACV']].groupby([FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds']).sum().rename(
    columns={'UNITS':'y'}).reset_index()

prev_row_cnt = forecastdf.shape[0]
forecastdf.dropna(subset=['y'], inplace=True) # Drop NAs
forecastdf = forecastdf.loc[forecastdf['y'] > 0] # Select strictly positive, non-zero values

"""
# Delete attribute-markets with low support = less than 50 rows by market & attribute 
# This is fine to ignore for now as e.g. Herbs will have representation in other markets
rowcounts = forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'MARKET']).count().reset_index()
rows_to_delist = rowcounts.loc[rowcounts['y'] < 52,[FOCUS_ATTRIBUTE_COLUMN,'MARKET']]
# Convert to str then join & select rows not matching rows_to_delist
forecastdf = forecastdf.loc[~(forecastdf[FOCUS_ATTRIBUTE_COLUMN].astype(str) + forecastdf['MARKET'].astype(str)).isin(
    rows_to_delist[FOCUS_ATTRIBUTE_COLUMN].astype(str) + rows_to_delist['MARKET'].astype(str))]
"""
current_row_cnt = forecastdf.shape[0]

print(f'Dropped {current_row_cnt - prev_row_cnt} rows with NA values. Final dataset has {current_row_cnt} rows')

Dropped 0 rows with NA values. Final dataset has 22568 rows


In [26]:
# Run as is
# Summary of the data rows by market & unique time
attr_sparsity_report = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index().pivot(
    index='Retailer', columns=FOCUS_ATTRIBUTE_COLUMN, values='ds')

num_unique_timeperiods = forecastdf['ds'].nunique()
for col in attr_sparsity_report.columns:
    attr_sparsity_report[col] = attr_sparsity_report[col]/num_unique_timeperiods

#attr_sparsity_report.to_excel('./attr_sparsity_report_'+FOCUS_ATTRIBUTE_COLUMN+'.xlsx')
attr_sparsity_report.describe()

SEG_DESC_ATTRIBUTE,AO SEGMENTS,COLA - LIGHT - COCA COLA & AO BRANDS,COLA - LIGHT - EIGEN MERK,COLA - LIGHT - PEPSI,COLA - REGULAR - COCA COLA,COLA - REGULAR - EIGEN MERK,COLA - REGULAR - PEPSI & AO BRANDS,COLA - ZERO - COCA COLA,ENERGY - AO BRANDS,ENERGY - RED BULL,ICE_TEA_BLACK - CAN,ICE_TEA_BLACK - CARTON,ICE_TEA_BLACK - PET MULTISERVE,ICE_TEA_BLACK - PET SINGLESERVE,ICE_TEA_GREEN - CAN,ICE_TEA_GREEN - CARTON,ICE_TEA_GREEN - PET MULTISERVE,ICE_TEA_GREEN - PET SINGLESERVE,LEMON_LIME,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,MINERAALWATER - PET SINGLESERVE,ORANGE,OV.GAZEUSES - LIGHT - AO BRANDS,OV.GAZEUSES - LIGHT - CRYSTAL CL,OV.GAZEUSES - LIGHT - EIGEN MERK,OV.GAZEUSES - LIGHT - SPA,OV.GAZEUSES - REGULAR - AO BRANDS,OV.GAZEUSES - REGULAR - FERNANDES,OV.GAZEUSES - REGULAR - RIVELLA,OV.VRUCHT.LIM. - LIGHT,OV.VRUCHT.LIM. - REGULAR - AO BRANDS,OV.VRUCHT.LIM. - REGULAR - EIGEN MERK,OV.VRUCHT.LIM. - REGULAR - HERO,OV.VRUCHT.LIM. - REGULAR - SPA,SPORT,TONIC,VRUCHT.LIM. - LIGHT,VRUCHT.LIM. - REGULAR - AO BRANDS,VRUCHT.LIM. - REGULAR - DUBBEL FRISS,VRUCHT.LIM. - REGULAR - ROOSVICEE,VRUCHTENDRANK - AO BRANDS,VRUCHTENDRANK - APPELSIENTJE,VRUCHTENDRANK - EIGEN MERK
count,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
mean,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00
std,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
min,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.93,1.00,1.00,1.00,1.00,1.00,1.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [27]:
# Run as is
prev_row_cnt = forecastdf.shape[0]

tt = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index()
tt.loc[:,'share_of_total_ds_with_data'] = tt.loc[:,'ds']/num_unique_timeperiods
drop_threshold = 0.5 # This is equivalent to 1yr if input is 2yrs worth of data. If only 1yr of data then set to 1
rows_to_keep = (tt.loc[tt['share_of_total_ds_with_data'] >= drop_threshold, 'Retailer'].str.cat(tt.loc[tt['share_of_total_ds_with_data'] >= drop_threshold, FOCUS_ATTRIBUTE_COLUMN])).tolist() # Drop rows will be those with < 1 yr of data to forecast with + nas
forecastdf = forecastdf.loc[forecastdf['Retailer'].str.cat(forecastdf[FOCUS_ATTRIBUTE_COLUMN]).isin(rows_to_keep)]

current_row_cnt = forecastdf.shape[0]

print(f'Dropped {current_row_cnt - prev_row_cnt} rows whose markets/attributes had insufficient data for forecasting. \nFinal dataset has {current_row_cnt} rows')



Dropped 0 rows whose markets/attributes had insufficient data for forecasting. 
Final dataset has 22568 rows


In [28]:
# Run as is
# Determine attributes to merge
"""
# Grouping Approach A -  Count of time period approach
# Calculate the % of Markets where the attribute is NaN i.e. Doesn't exist
count_sparsity = pd.DataFrame(attr_sparsity_report.count()/attr_sparsity_report.count().max())
mean_timeperiods_sparsity = pd.DataFrame(attr_sparsity_report.mean())
merge_threshold = 0.5 #40%

# Merge any attributes that are missing data in less than merge_threshold market/time_periods
attributes_to_group = count_sparsity.loc[count_sparsity[0] < merge_threshold].merge(
    mean_timeperiods_sparsity.loc[mean_timeperiods_sparsity[0] < merge_threshold], 
    on=FOCUS_ATTRIBUTE_COLUMN, how='outer').index.to_list()
"""
# Grouping Approach B -  Share of y
sparsity_y_based = forecastdf[[FOCUS_ATTRIBUTE_COLUMN,'y']].groupby([FOCUS_ATTRIBUTE_COLUMN]).agg(y_sum=("y","sum")).reset_index()
sparsity_y_based['y_share'] = sparsity_y_based['y_sum']/sparsity_y_based['y_sum'].sum()*100

try:
    attributes_to_group = sparsity_y_based.loc[sparsity_y_based['y_share'] < 0.05, FOCUS_ATTRIBUTE_COLUMN].unique().to_list()
    print(f"Merging the attributes below into an ALL OTHER group due to their low support: \n\n {attributes_to_group}")
except AttributeError: #empty list returned
    attributes_to_group = []
    print("There were no attributes to merge")
                                           

There were no attributes to merge


In [29]:
# Run as is
# Groups attributes if they have been flagged for merging
grouped_attributes_forecastdf = forecastdf.loc[forecastdf[FOCUS_ATTRIBUTE_COLUMN].isin(attributes_to_group)].groupby(['Retailer','ds']).sum().dropna(how='all').reset_index()
grouped_attributes_forecastdf[FOCUS_ATTRIBUTE_COLUMN] = 'ALL_OTHER_'+FOCUS_ATTRIBUTE_COLUMN

# Drop all rows which have been grouped then concatenate with the new grouped rows
forecastdf = pd.concat([forecastdf.loc[~forecastdf[FOCUS_ATTRIBUTE_COLUMN].isin(attributes_to_group)].reset_index(drop=True), grouped_attributes_forecastdf], 
          axis=0, ignore_index=True, sort=False).sort_values(by=['Retailer',FOCUS_ATTRIBUTE_COLUMN,'ds']).reset_index(drop=True)

# Summary of the data rows by market & unique time
attr_sparsity_report = forecastdf[['Retailer','ds',FOCUS_ATTRIBUTE_COLUMN]].groupby(['Retailer',FOCUS_ATTRIBUTE_COLUMN]).count().reset_index().pivot(
    index='Retailer', columns=FOCUS_ATTRIBUTE_COLUMN, values='ds')

num_unique_timeperiods = forecastdf['ds'].nunique()
for col in attr_sparsity_report.columns:
    attr_sparsity_report[col] = attr_sparsity_report[col]/num_unique_timeperiods

#attr_sparsity_report.to_excel('./attr_sparsity_report_'+FOCUS_ATTRIBUTE_COLUMN+'.xlsx')
attr_sparsity_report.describe()

SEG_DESC_ATTRIBUTE,AO SEGMENTS,COLA - LIGHT - COCA COLA & AO BRANDS,COLA - LIGHT - EIGEN MERK,COLA - LIGHT - PEPSI,COLA - REGULAR - COCA COLA,COLA - REGULAR - EIGEN MERK,COLA - REGULAR - PEPSI & AO BRANDS,COLA - ZERO - COCA COLA,ENERGY - AO BRANDS,ENERGY - RED BULL,ICE_TEA_BLACK - CAN,ICE_TEA_BLACK - CARTON,ICE_TEA_BLACK - PET MULTISERVE,ICE_TEA_BLACK - PET SINGLESERVE,ICE_TEA_GREEN - CAN,ICE_TEA_GREEN - CARTON,ICE_TEA_GREEN - PET MULTISERVE,ICE_TEA_GREEN - PET SINGLESERVE,LEMON_LIME,MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,MINERAALWATER - PET SINGLESERVE,ORANGE,OV.GAZEUSES - LIGHT - AO BRANDS,OV.GAZEUSES - LIGHT - CRYSTAL CL,OV.GAZEUSES - LIGHT - EIGEN MERK,OV.GAZEUSES - LIGHT - SPA,OV.GAZEUSES - REGULAR - AO BRANDS,OV.GAZEUSES - REGULAR - FERNANDES,OV.GAZEUSES - REGULAR - RIVELLA,OV.VRUCHT.LIM. - LIGHT,OV.VRUCHT.LIM. - REGULAR - AO BRANDS,OV.VRUCHT.LIM. - REGULAR - EIGEN MERK,OV.VRUCHT.LIM. - REGULAR - HERO,OV.VRUCHT.LIM. - REGULAR - SPA,SPORT,TONIC,VRUCHT.LIM. - LIGHT,VRUCHT.LIM. - REGULAR - AO BRANDS,VRUCHT.LIM. - REGULAR - DUBBEL FRISS,VRUCHT.LIM. - REGULAR - ROOSVICEE,VRUCHTENDRANK - AO BRANDS,VRUCHTENDRANK - APPELSIENTJE,VRUCHTENDRANK - EIGEN MERK
count,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
mean,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.99,1.00,1.00,1.00,1.00,1.00,1.00
std,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00
min,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.93,1.00,1.00,1.00,1.00,1.00,1.00
25%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
50%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
75%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [30]:
forecastdf = forecastdf.loc[forecastdf['y']>0]

This creates the forecasts by attribute value & market that we will use to factor out other external changes

This code takes a while depending on the factors above. Typical is 3-5 mins if you have a "reasonable" # of unique values ~ < 30

May take upwards of 30 mins for high cardinality attributes. I recommend clustering these before importing

In [31]:
@dask.delayed
def generate_prediction_delayed(dfpredict, myhols):
    #mdl = Prophet(seasonality_mode='multiplicative', seasonality_prior_scale=0.5)    
    ## add cap & floor for df & future
    mydf = dfpredict.copy()
    # Box-Cox transformation of Order 0
    mydf['y'] = np.log(1+mydf['y']) #log transform
    mydf['cap'] = mydf['y'].max()
    mydf['floor'] = 1
    
    mdl = Prophet(growth='logistic', changepoint_prior_scale=0.001, holidays=myhols, yearly_seasonality=True).fit(mydf, iter=100)
    
    # Create future dataset & forecast against
    future = mdl.make_future_dataframe(periods=52, freq='W')
    future['cap'] = mydf['y'].max()
    future['floor'] = 1
    #Maybe update hols here?

    # Predict
    forecast = mdl.predict(future)
    
    # Inverse Box-Cox
    forecast['yhat'] = np.exp(forecast['yhat']) - 1
    forecast['trend'] = np.exp(forecast['trend']) - 1

    # print("Calculating MAPE . . .")
    # df_mape = mapefn(dfpredict['y'], forecast['yhat'])

    # return mdl, df_mape
    return dask.delayed(forecast) # return delayed object since the function is expected to return a delayed
    


In [32]:
# Run as is
# Create dict of dfs which will each be individually forecasted by market & attribute
dictdfs = {k: v for k, v in forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'Retailer'])}
times_list = pd.to_datetime(forecastdf['ds'].to_list())
myhols = makeprophethols(times_list, future_periods=52, date_freq='W')
predictiondict = defaultdict(Prophet)
forecastdict = defaultdict(pd.DataFrame)

In [33]:
# Run as is

forecast_holder = []


for df_key, dfpredict in dictdfs.items():
    dfpredict_delayed = dask.delayed(dfpredict)
    forecast_holder.append(generate_prediction_delayed(dfpredict_delayed, myhols))
    #forecast_holder['MARKET'] = df_key
    #print(f"Type: {type(forecastdict[df_key])}")

#forecast_holder.append(forecastdf.groupby([FOCUS_ATTRIBUTE_COLUMN,'MARKET']).apply(generate_prediction_delayed(dfpredict_delayed, myhols)))
# This works but I don't understand why I have to compute twice (here and cell below) given how costly the function is
forecasts = dask.delayed(forecast_holder).compute()
forecasts

[Delayed('DataFrame-5872c183-41b6-489f-bf85-d5f1ba479725'),
 Delayed('DataFrame-db16a3c9-eb70-410d-a00c-22200c671f79'),
 Delayed('DataFrame-419aa3c1-d78d-4869-ad41-fc1ab89f783a'),
 Delayed('DataFrame-acedddfa-21da-4e7a-9fdd-ed5b431d3fc6'),
 Delayed('DataFrame-829051d6-113f-4fb9-b81a-4648a3123062'),
 Delayed('DataFrame-c34279e3-ec2e-46f2-a914-d4ddf7a6c778'),
 Delayed('DataFrame-af753ebd-512e-4941-929b-da9796e4e3af'),
 Delayed('DataFrame-f940fe0e-e4a3-46b2-855a-906658fd7f61'),
 Delayed('DataFrame-3c5c1817-4f90-45cf-b385-6ec807e78762'),
 Delayed('DataFrame-b12a43d7-0510-4b72-93e7-ff2969e9cf14'),
 Delayed('DataFrame-d6d44774-15a0-41f7-b100-9784dcd4feb4'),
 Delayed('DataFrame-a62fa4ce-b06a-4fc9-8b7d-dc3bdd5131dc'),
 Delayed('DataFrame-69a8bcde-02c1-430e-a729-cd2cf5208688'),
 Delayed('DataFrame-f26e0903-9d09-4268-836c-12256727688f'),
 Delayed('DataFrame-c2f6747a-e1f9-4cd4-9694-edd2662e94a6'),
 Delayed('DataFrame-91d1535c-7079-4335-bc3a-317db1fc9fb6'),
 Delayed('DataFrame-c9ba2a15-235f-4f1f-a

In [34]:
# Run as is
# y = the metric of focus -> typically should be BASE_UNITS
# yhat = forecasted value for y in that attribute, period & market
# catsales_nonforecast = y - yhat

dfcatforecast = pd.DataFrame()
for i in range(len(dictdfs.keys())):
    tempdf = forecasts[i].compute()
    tempdf[FOCUS_ATTRIBUTE_COLUMN], tempdf['Retailer'] = list(dictdfs.keys())[i]
    dfcatforecast = dfcatforecast.append(tempdf)
    
forecastdf = forecastdf.merge(dfcatforecast[[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds','yhat','trend']], 
                              how='left', on=[FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds'], suffixes=("","_y"))
#forecastdf['catsales_nonforecast'] = forecastdf['y'] - forecastdf['yhat']
forecastdf['catsales_nonforecast'] = forecastdf['y'] - forecastdf['trend']

forecastdf[['y','yhat','trend','catsales_nonforecast']].describe()

,y,yhat,trend,catsales_nonforecast
count,"22,568.00","22,568.00","22,568.00","22,568.00"
mean,"76,795.91","78,013.34","76,636.97",158.94
std,"126,295.02","118,217.66","114,630.49","48,437.22"
min,1.00,0.98,43.42,"-391,615.84"
25%,"9,474.50","11,673.68","11,450.72","-8,518.03"
50%,"30,372.00","35,835.47","35,157.73","-1,060.26"
75%,"86,569.75","85,993.62","85,614.54","4,837.10"
max,"1,174,931.00","912,534.13","761,543.10","549,550.43"


In [35]:
# Run as is
forecastdf_filtered = forecastdf.copy()
forecastdf_filtered['catsales_nonforecast_perc'] = forecastdf_filtered['catsales_nonforecast']/forecastdf_filtered['y']
# If deviation is <1%, set to np.nan
forecastdf_filtered.loc[abs(forecastdf_filtered['catsales_nonforecast_perc']) < 0.01, 'catsales_nonforecast'] = np.nan 
forecastdf_filtered.drop('catsales_nonforecast_perc', axis=1, inplace=True)
# Check for % of sales that are not accounted for by forecasted trend & seasonality
pd.DataFrame(forecastdf_filtered['catsales_nonforecast']/forecastdf_filtered['y']).describe()


,0
count,"21,922.00"
mean,-0.42
std,2.06
min,-124.98
25%,-0.40
50%,-0.07
75%,0.14
max,0.96


In [36]:
# Run as is
# Merge category forecast to original dataset & create regression base dataset
#forecastdf_new = createfile(forecastdf_filtered, valcol='catsales_nonforecast', tdpcol='ACV').dropna(how='all')
forecastdf_new = createfile(forecastdf_filtered, valcol='y', tdpcol='ACV').dropna(how='all')
# "Saves" value of focus attribute in a new column as this will be removed by get_dummies
forecastdf_new.loc[:,FOCUS_ATTRIBUTE_COLUMN+'2'] = forecastdf_new.loc[:,FOCUS_ATTRIBUTE_COLUMN].copy()

metriccols= forecastdf_new.columns[forecastdf_new.columns.str.contains('perc')].to_flat_index().to_list()
dummycols = list(FOCUS_ATTRIBUTE_COLUMN + "_"+ forecastdf_new[FOCUS_ATTRIBUTE_COLUMN].unique())
forecastdf_dummy = pd.get_dummies(forecastdf_new, columns=[FOCUS_ATTRIBUTE_COLUMN])
for col in dummycols:
    for col2 in metriccols:
        forecastdf_dummy[col2 +'_' +col] = forecastdf_dummy[col] * forecastdf_dummy[col2]
        
# Restores focus attribute column
forecastdf_dummy.rename(columns={FOCUS_ATTRIBUTE_COLUMN+'2':FOCUS_ATTRIBUTE_COLUMN}, inplace=True)

# Create regression aggregated dataset - By Market & Time Period
forecastdf_dummy.replace(0, np.nan, inplace=True) #Prevent zeros from messing the means
forecastdf_dummy_agg = forecastdf_dummy.loc[:,
                                            [FOCUS_ATTRIBUTE_COLUMN,'Retailer','ds'] + 
                                            forecastdf_dummy.columns[
                                                forecastdf_dummy.columns.str.contains('perc') & 
                                                # ensure line below has your attribute name
                                                forecastdf_dummy.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN) &
                                                ~forecastdf_dummy.columns.str.contains('4wk')
                                            ].to_flat_index().to_list()].groupby(['Retailer','ds']).mean().reset_index()


forecastdf_dummy_agg.describe()

C:\Users\TRAINO~1\AppData\Local\Temp/ipykernel_22928/1646407835.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  forecastdf_dummy[col2 +'_' +col] = forecastdf_dummy[col] * forecastdf_dummy[col2]


,val_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - PEPSI & AO BRANDS,...,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - LIGHT,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK
count,523.00,524.00,522.00,522.00,524.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,524.00,...,518.00,525.00,525.00,525.00,525.00,525.00,523.00,525.00,524.00,524.00,512.00,525.00,516.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,525.00,522.00,525.00,524.00
mean,0.17,0.00,0.06,0.00,0.02,-0.00,-0.01,-0.01,0.05,0.02,0.06,0.03,0.18,-0.00,0.00,0.00,0.07,0.01,0.10,0.01,0.01,-0.00,-0.01,-0.00,0.09,...,0.10,0.07,0.02,0.05,0.03,0.57,-0.00,0.09,-0.01,0.23,0.00,0.14,0.00,0.22,0.01,0.03,0.02,0.37,-0.00,0.08,-0.00,0.02,0.01,0.03,0.01
std,0.81,0.06,1.17,0.09,0.22,0.04,0.26,0.07,0.35,0.15,0.53,0.22,0.79,0.07,1.06,0.10,0.60,0.09,0.77,0.11,0.18,0.05,0.24,0.08,0.54,...,1.14,0.45,0.21,0.75,0.29,1.65,0.09,2.51,0.13,1.38,0.10,1.84,0.12,0.90,0.17,1.29,0.23,1.18,0.04,1.88,0.07,0.22,0.08,0.29,0.11
min,-0.94,-0.30,-3.74,-0.

The next code cell filters down the regression dataset by filtering outliers.

Change the 'perc_limit' variable below only if your 25%/75% percentile values are greater than 3 in absolute terms (Use the Guideline comment). 

Else run as-is

Finally, compare the Count row to see how many rows have been filtered out and adjust accordingly

In [37]:
# Filter out extreme swings in non-forecasted sales i.e. greater than 300% change (likely due to some Promo effect)

# Select only indexes where each column value is between min & max
perc_limit = 1.5 # Guideline: Set to 1 + absolute value of (max of 25%/75% percentiles)

"""
# ------ APPROACH 1: REQUIRES PERFECT MATCHING. DOES NOT SCALE WELL IF  ATTRIBUTE HAS > 20 UNIQUE VALUES
# Only works if the numeric cols are the same scale e.g. Percentages
chosen_idx_list = []
for col in forecastdf_dummy_agg.select_dtypes('number').columns:
    chosen_idx_list.append(pd.DataFrame(forecastdf_dummy_agg[abs(forecastdf_dummy_agg[col]) <= perc_limit].index, index=forecastdf_dummy_agg[abs(forecastdf_dummy_agg[col]) <= perc_limit].index))
# Selects only the indices that match across all the columns
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg.iloc[
    pd.concat([k for k in chosen_idx_list], axis=1, join='inner').iloc[:,0].index]
"""

# ------ APPROACH 2: REPLACE WITH NA WHICH REGRESSION MODEL WILL IMPUTE
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg.copy()
for col in forecastdf_dummy_agg.select_dtypes('number').columns:
    forecastdf_dummy_agg_filtered.loc[abs(forecastdf_dummy_agg_filtered[col]) > perc_limit, col] = np.nan
    
# Drop rows where all values are NA
forecastdf_dummy_agg_filtered.dropna(axis=0, how='all')
#Interpolate NA rows based on time series
forecastdf_dummy_agg_filtered = forecastdf_dummy_agg_filtered.sort_values(by=['Retailer','ds']).set_index('ds').interpolate(method='time').reset_index().dropna()
forecastdf_dummy_agg_filtered.describe()

,val_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - PEPSI & AO BRANDS,...,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - LIGHT,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK
count,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,...,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00,522.00
mean,0.02,0.00,-0.03,0.00,0.01,-0.00,-0.01,-0.01,0.03,0.01,0.02,0.02,0.02,-0.00,-0.02,-0.00,0.03,0.01,0.01,0.01,0.01,-0.00,-0.00,-0.00,0.01,...,0.02,0.04,-0.00,-0.00,-0.01,-0.10,-0.00,0.06,-0.00,-0.03,-0.00,-0.02,-0.00,-0.04,-0.00,0.02,0.00,-0.07,-0.00,-0.00,-0.00,0.01,0.00,0.00,0.01
std,0.43,0.06,0.52,0.09,0.21,0.04,0.25,0.07,0.22,0.12,0.29,0.18,0.47,0.06,0.53,0.09,0.25,0.09,0.33,0.11,0.18,0.05,0.23,0.07,0.37,...,0.17,0.33,0.03,0.40,0.05,0.54,0.08,0.59,0.12,0.32,0.05,0.36,0.09,0.38,0.06,0.37,0.09,0.48,0.04,0.53,0.07,0.13,0.07,0.13,0.10
min,-0.94,

In [38]:
# Run as is
perc_rows_selected = forecastdf_dummy_agg_filtered['ds'].count() / forecastdf_dummy_agg['ds'].count()
print(f"Selected {perc_rows_selected*100: .2f}% of the dataset rows for regression \n")
print(f"as these have less than a {perc_limit*100: .2f}% swing in value vs. prior period")

Selected  99.43% of the dataset rows for regression 

as these have less than a  150.00% swing in value vs. prior period


In [39]:
# Run as is
# Create actual regression datasets
regression_df = forecastdf_dummy_agg_filtered.loc[:, 
                                     ['Retailer','ds'] + 
                                     forecastdf_dummy_agg_filtered.columns[
                                         forecastdf_dummy_agg_filtered.columns.str.contains('perc') & 
                                         # ensure line below has your attribute name
                                         forecastdf_dummy_agg_filtered.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN)
                                     ].to_flat_index().to_list()].reset_index(drop=True)

# Split datasets
regression_df = regression_df.loc[:,~regression_df.columns.duplicated()] # Drop duplicate columns

## Categorical column transformations
# Encodes the categorical columns as strings so they can work with xgboost
regression_df['ds_month'] = regression_df['ds'].dt.month
regression_df['ds_year'] = regression_df['ds'].dt.year
# Note: LabelEncoder encodes a cardinal order e.g. 1>2>3 - Relevant for dates
lbl_month = LabelEncoder()
regression_df['ds_month'] = lbl_month.fit_transform(regression_df['ds_month'].astype(str))
lbl_year = LabelEncoder()
regression_df['ds_year'] = lbl_year.fit_transform(regression_df['ds_year'].astype(str))
# Onehot encode Market label & drops it from set
regression_df = pd.get_dummies(regression_df, columns=['Retailer'])  

# Create train & test dfs
regression_df = regression_df.drop('ds', axis=1)
regression_df_train = regression_df.sample(frac=0.8, random_state=42) # Selects 80% of dataset for main training
regression_df_test = regression_df.drop(regression_df_train.index) # Selects remainder of dataset for sight-unseen testing
regression_df_train.reset_index(drop=True, inplace=True)
regression_df_test.reset_index(drop=True, inplace=True)
regression_df_train

,val_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - PEPSI & AO BRANDS,...,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,val_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,dist_chg2_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK,ds_month,ds_year,Retailer_Albert Heijn,Retailer_Hoogvliet,Retailer_Jumbo,Retailer_Plus,Retailer_Vomar
0,0.06,-0.02,0.00,-0.08,0.01,-0.01,0.04,-0.03,0.22,-0.07,-0.32,0.09,1.06,-0.01,-0.03,-0.10,0.01,-0.00,0.01,-0.02,0.20,0.13,-0.10,0.16,0.19,...,-0.01,-0.02,0.15,-0.01,0.12,-0.01,0.02,-0.03,0.19,-0.07,0.02,-0.02,0.06,-0.02,0.02,-0.09,0.10,-0.00,4,1,0,0,0,0,1
1,-0.60,-0.07,-0.64,-0.05,-0.03,0.02,0.03,-0.01,0.09,-0.01,-0.55,-0.01,0.69,0.03,-0.59,0.10,-0.24,-0.00,-0.03,-0.02,0.07,-0.01,-0.36,-0.01,0.52,...,-0.85,-0.01,-0.02,-0.02,0.02,-0.05,0.01,-0.01,-0.00,-0.03,-0.02,0.01,0.09,-0.00,0.00,0.01,-0.03,0.00,5,1,0,0,0,0,1
2,-0.02,-0.01,0.01,-0.01,-0.25,-0.08,-0.30,-0.09,0.04,0.00,0.01,-0.00,0.02,0.00,0.01,0.00,-0.28,-0.03,-0.19,-0.03,0.03,-0.00,-0.00,0.00,-0.26,...,0.82,0.00,-0.08,-0.03,0.05,-0.08,0.18,-0.05,-0.17,-0.10,-0.40,-0.03,-0.02,-0.08,0.01,-0.05,0.02,-0.11,3,0,1,0,0,0,0
3,0.09,0.03,0.04,0.02,-0.12,-0.01,0.02,-0.04,0.06,0.09,-0.01,0.08,1.23,0.02,0.27,0.05,-0.09,-0.00,-0.02,-0.04,0.01,0.02,-0.00,0.01,1.47,...,0.21,-0.00,-0.26,0.00,0.08,0.01,-0.10,-0.01,-0.02,-0.00,0.22,-0.01,0.03,-0.01,-0.08,-0.03,-0.13,-0.03,7,2,0,1,0,0,0
4,-0.11,0.01,-0.07,-0.00,-0.12,0.03,-0.32,0.01,-0.05,0.01,-0.04,0.00,0.88,0.02,1.02,0.12,0.08,0.11,-0.03,0.10,-0.03,0.01,-0.03,-0.00,0.69,...,-0.16,-0.01,0.04,0.01,0.01,-0.01,-0.33,0.04,-0.13,0.03,-0.35,0.01,-0.21,-0.01,-0.15,0.22,-0.27,0

We will predict the transferable demand for each of these columns below in turn

In [40]:
# Run as is
prediction_cols = regression_df_train.columns[
    (regression_df_train.columns.str.contains('val_chg1')) & 
    (regression_df_train.columns.str.contains(FOCUS_ATTRIBUTE_COLUMN))].to_flat_index().to_list()
regression_df_train[prediction_cols].describe()

,val_chg1_perc_SEG_DESC_ATTRIBUTE_AO SEGMENTS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - COCA COLA & AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - LIGHT - PEPSI,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - COCA COLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - REGULAR - PEPSI & AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_COLA - ZERO - COCA COLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENERGY - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_ENERGY - RED BULL,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_BLACK - CAN,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_BLACK - CARTON,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_BLACK - PET MULTISERVE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_BLACK - PET SINGLESERVE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_GREEN - CAN,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_GREEN - CARTON,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_GREEN - PET MULTISERVE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ICE_TEA_GREEN - PET SINGLESERVE,val_chg1_perc_SEG_DESC_ATTRIBUTE_LEMON_LIME,val_chg1_perc_SEG_DESC_ATTRIBUTE_MINERAALWATER - PET MULTISERVE & AO PACK FORMATS,val_chg1_perc_SEG_DESC_ATTRIBUTE_MINERAALWATER - PET SINGLESERVE,val_chg1_perc_SEG_DESC_ATTRIBUTE_ORANGE,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - LIGHT - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - LIGHT - CRYSTAL CL,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - LIGHT - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - LIGHT - SPA,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - REGULAR - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - REGULAR - FERNANDES,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.GAZEUSES - REGULAR - RIVELLA,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.VRUCHT.LIM. - LIGHT,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.VRUCHT.LIM. - REGULAR - EIGEN MERK,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.VRUCHT.LIM. - REGULAR - HERO,val_chg1_perc_SEG_DESC_ATTRIBUTE_OV.VRUCHT.LIM. - REGULAR - SPA,val_chg1_perc_SEG_DESC_ATTRIBUTE_SPORT,val_chg1_perc_SEG_DESC_ATTRIBUTE_TONIC,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - LIGHT,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - DUBBEL FRISS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHT.LIM. - REGULAR - ROOSVICEE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - AO BRANDS,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - APPELSIENTJE,val_chg1_perc_SEG_DESC_ATTRIBUTE_VRUCHTENDRANK - EIGEN MERK
count,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00,418.00
mean,0.02,0.01,0.02,0.03,0.03,0.01,0.03,0.05,0.01,0.01,-0.01,0.06,-0.01,0.06,0.00,0.09,-0.00,0.05,0.05,0.02,0.01,0.02,0.03,-0.03,0.01,0.03,0.00,0.01,0.03,0.06,0.04,0.01,-0.02,-0.04,0.02,0.07,-0.05,0.04,-0.11,-0.04,-0.04,-0.07,0.01
std,0.43,0.21,0.23,0.47,0.24,0.18,0.37,0.29,0.09,0.20,0.37,0.39,0.48,0.41,0.34,0.51,0.51,0.39,0.28,0.15,0.14,0.19,0.34,0.42,0.12,0.44,0.14,0.36,0.49,0.33,0.29,0.13,0.32,0.38,0.17,0.43,0.49,0.33,0.53,0.32,0.39,0.48,0.12
min,-0.80,-0.55,-0.60,-0.83,-0.58,-0.79,-0.75,-0.66,-0.36,-0.80,-0.99,-0.74,-0.83,-0.81,-0.99,-0.73,-0.89,-0.76,-0.61,-0.32,-0.44,-0.46,-0.90,-0.86,-0.54,-0.82,-0.42,-0.82,-0.82,-0.64,-0.71,-0.81,-0.88,-0.90,-0.60,-0.74,-1.00,-0.84,-0.92,-0.90,-0.84,-0.89,-0.38
25%,-0.16,-0.12,-0.06,-0.22,-0.10,-0.06,-0.16,-0.11,-0.04,-0.07,-0.17,-0.20,-0.38,-0.14,-0.16,-0.25,-0.41,-0.15,-0.11,-0.05,-0.06,-0.10,-0.12,-0.17,-0.05,-0.14,-0.08,-0.09,-0.23,-0.11,-0.11,-0.05,-0.08,-0.15,-0.08,-0.14,-0.42,-0.13,-0.57,-0.09,-0.15,-0.47,-0.04
50%,-0.00,-0.01,0.00,-0.01,0.00,0.01,-0.00,0.01,-0.00,-0.01,-0.01,0.01,-0.01,0.02,-0.01,0.01,-0.03,0.01,0.02,0.01,0.00,0.01,-0.01,0.01,0.

In [41]:
# Run as is
## Bayes Robust Approach
final_coefs_dict = {} # defaultdict(pd.DataFrame)
final_scores_dict = {}

for col in prediction_cols:
    my_param_grid = {
        'tol':[1e-3, 1e-4],
        'alpha_1':[1e-6, 1e-8, 1e-4],
        'lambda_init':[1, 0.5, 10]    
    }
    reg_train_pipe = Pipeline([
        ('transformer', QuantileTransformer(output_distribution='normal')),
        ('model', GridSearchCV(BayesianRidge(n_iter=500, fit_intercept=True), 
                                    param_grid=my_param_grid, scoring='r2', n_jobs=-1))
    ])
    clf = reg_train_pipe.fit(regression_df_train.drop(col, axis=1), regression_df_train.loc[:,col])
    myscore = clf.score(regression_df_test.drop(col, axis=1), regression_df_test.loc[:,col])
    final_scores_dict[col] = pd.DataFrame(pd.Series(myscore, index=[col],name='Score'))
    temp_coefs_df = pd.DataFrame(list(zip(regression_df_train.drop(col, axis=1).columns, clf['model'].best_estimator_.coef_)), columns=['Feature','Coefficient'])
    # Select only the dist_chg numbers
    final_coefs_dict[col] = temp_coefs_df.loc[
        temp_coefs_df['Feature'].str.contains(FOCUS_ATTRIBUTE_COLUMN) & 
        temp_coefs_df['Feature'].str.contains('dist_chg1')].copy()
 

C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_

C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\trainor-gilhamt\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (418). n_

In [42]:
final_coefs = pd.DataFrame()
final_scores = pd.DataFrame()

for k in final_coefs_dict.keys():
    final_coefs_dict[k].loc[:,'TO_ATTRIBUTE'] = k # Set the key as a column in the dataframe
    final_coefs = final_coefs.append(final_coefs_dict[k].copy(), ignore_index=True)
    
    final_scores_dict[k].loc[:,'MODEL_NAME'] = k # Set the key as a column in the dataframe
    final_scores = final_scores.append(final_scores_dict[k])

# Create final accuracy scores
final_scores['MODEL_NAME'] = final_scores['MODEL_NAME'].str.replace("val_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_","")
final_scores_bayes = final_scores.copy()

# Format & adjust coefs
final_coefs.rename(columns={'Feature':'FROM_ATTRIBUTE'}, inplace=True)

final_coefs['TO_ATTRIBUTE'] = final_coefs['TO_ATTRIBUTE'].str.replace("val_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_","")
final_coefs['FROM_ATTRIBUTE'] = final_coefs['FROM_ATTRIBUTE'].str.replace("dist_chg1_perc_","").str.replace(FOCUS_ATTRIBUTE_COLUMN+"_",'')

"""
# Keep self interaction by ensuring it is negative i.e. substitute
#final_coefs.loc[(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE']) & 
#                (final_coefs['Coefficient']>0), 'Coefficient'] = final_coefs.loc[(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE']) & 
#                                                                                 (final_coefs['Coefficient']>0), 'Coefficient'] * -1
# For ISD, remove self elasts
final_coefs = final_coefs.loc[~(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE'])]
                              
# Zero out positive coefficients i.e. Increase Dist -> Increase value: Implies complements vs. substitutes
final_coefs.loc[final_coefs['Coefficient'] > 0, 'Coefficient'] = 0

# Adjust final coefficients for export

# Rebase % transfers to 100%
final_coefs_adjusted_bayes = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient']].groupby(['TO_ATTRIBUTE']).sum(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_sum")
)

final_coefs_adjusted_bayes.loc[:, 'Percent_Transfer'] = final_coefs_adjusted_bayes['Coefficient'] / final_coefs_adjusted_bayes['Coefficient_sum']
final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False)
"""
# For ISD, remove self elasts
final_coefs = final_coefs.loc[~(final_coefs['TO_ATTRIBUTE'] == final_coefs['FROM_ATTRIBUTE'])]

# Invert the scale so the substitutes (-ve coeff) are now positives
final_coefs.loc[:,'Coefficient'] = final_coefs.loc[:,'Coefficient'] *-1

# Finalise Percent Transfer by abs-value shift to account for -ve coefficients
# Note: Complements i.e. highly negative coeffs (post inversion) will be close to zero and thus regularised naturally
final_coefs = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient']].groupby(['TO_ATTRIBUTE']).min(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_min")
)

final_coefs['Coefficient_minabs'] = abs(final_coefs['Coefficient_min'])
final_coefs['Coefficient_pos_shift'] = final_coefs['Coefficient'] + final_coefs['Coefficient_minabs']

final_coefs_adjusted_bayes = final_coefs.merge(
    final_coefs[['TO_ATTRIBUTE','Coefficient_pos_shift']].groupby(['TO_ATTRIBUTE']).sum(), 
    on=['TO_ATTRIBUTE'], suffixes=("","_sum")
)

final_coefs_adjusted_bayes.loc[:, 'Percent_Transfer'] = final_coefs_adjusted_bayes['Coefficient_pos_shift'] / final_coefs_adjusted_bayes['Coefficient_pos_shift_sum']
final_coefs_adjusted_bayes = final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False)


final_coefs_adjusted_bayes

,TO_ATTRIBUTE,FROM_ATTRIBUTE,Percent_Transfer
1791,VRUCHTENDRANK - EIGEN MERK,OV.GAZEUSES - REGULAR - FERNANDES,0.04
1773,VRUCHTENDRANK - EIGEN MERK,ENERGY - RED BULL,0.04
1788,VRUCHTENDRANK - EIGEN MERK,OV.GAZEUSES - LIGHT - EIGEN MERK,0.04
1795,VRUCHTENDRANK - EIGEN MERK,OV.VRUCHT.LIM. - REGULAR - EIGEN MERK,0.04
1770,VRUCHTENDRANK - EIGEN MERK,COLA - REGULAR - PEPSI & AO BRANDS,0.04
...,...,...,...
10,AO SEGMENTS,ICE_TEA_BLACK - CARTON,0.01
13,AO SEGMENTS,ICE_TEA_GREEN - CAN,0.01
1,AO SEGMENTS,COLA - LIGHT - EIGEN MERK,0.00
9,AO SEGMENTS,ICE_TEA_BLACK - CAN,0.00


In [43]:
final_coefs_adjusted_bayes[['TO_ATTRIBUTE','FROM_ATTRIBUTE','Percent_Transfer']].sort_values(['TO_ATTRIBUTE','Percent_Transfer'], ascending=False).to_excel('./'+ANALYSIS_CATEGORY+' ATTR_SPECIFIC_BAYES_'+FOCUS_ATTRIBUTE_COLUMN+'_COEFS.xlsx', index=False)
final_scores_bayes.sort_values('MODEL_NAME', ascending=True).to_excel('./'+ANALYSIS_CATEGORY+' ATTR_SPECIFIC_BAYES_'+FOCUS_ATTRIBUTE_COLUMN+'_SCORES.xlsx', index=False)

YOU'RE DONE!!!

Percent Transfers have been outputted to file that starts with your Category Name & Attribute of Focus in this directory